In [1]:
#Import libraries
import bs4
import requests
import pandas as pd 
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\APOORV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\APOORV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Scrape data

In [2]:
issues = ['1906','1905','1904','1903','1902','1901',
          '1806','1805','1804','1803','1802','1801',
          ]
data = pd.DataFrame(columns = ['Issue', 'Article Name', 'Author', 'Content'])
for f in issues:
    url = "https://hbr.org/archive-toc/BR"+f
    cpage = requests.get(url)
    csoup = bs4.BeautifulSoup(cpage.content, 'lxml')

    #Find links to all the articles

    raw = csoup.find_all('h3', {'class':'hed'})
    raw = set(raw)
    s = []
    for i in raw:
        try:
            a = i.find('a').attrs['href']
            s.append(a)
        except:
            m = 1
    for i in range(len(s)):
        s[i] = 'http://hbr.org'+s[i]


    #Iterate throught the articles for the issue


    
    k = -1
    for i in s:
        page = requests.get(i)
        soup = bs4.BeautifulSoup(page.content, 'lxml')

        content = soup.find(name = 'article-content')
        try:
            title = content.find('h1', {'class':'article-hed'}).text
        except:
            title = content.find_all('h2', {'class':'article-hed'})[k].text
            if len(content.find_all('h2', {'class':'article-hed'}))>1:
                    k+=1

        try:
            author = content.find('div').find('div').find('div').find('div', {"class": "byline"}).find('ul').find('li').text
        except:
            author = ''
        contents = ''
        body = content.find('div', {'article article-first-row premium'})
        if(body==None):
            try:
                body = content.find_all('div', {'class':'article article-first-row'})[k]
            except:
                body = content.find_all('div', {'class':'article article-first-row'})
        body = body.find_all('p')
        for j in body:
            contents+= j.text + ' '
        try:
            issue = body.find('div', {'class': 'mvm description-text hbrgray, line-height-tight'}).find('a').text
        except:
            m = 2
        s2 = pd.Series([f, title, author, contents], index=['Issue', 'Article Name', 'Author', 'Content'])
        data = data.append(s2, ignore_index=True)

In [3]:
data

,Issue,Article Name,Author,Content
0,1906,The Truth About CEO Tenure,Adi Ignatius,The leaders on our 2019 list of the world’s b...
1,1906,The Secrets of Successful Female Networkers,,One oft-cited reason why more female executive...
2,1906,The Leader as Coach,,"To cope with disruptive change, companies are ..."
3,1906,How the Best Bosses Interrupt Bias on Their Teams,Joan C. Williams,Companies spend millions on antibias training ...
4,1906,The CEO’s Guide to Retirement,,"When Jensen Huang cofounded NVIDIA, in 1993, h..."
...,...,...,...,...
228,1801,Finding Your Company’s Second Act,,Why do so few companies achieve successful sec...
229,1801,What’s Your Organization’s Cultural Profile?,,First you must identify culture targets. The b...
230,1801,CEOs Step into the Fray,Adi Ignatius,"Once upon a time, CEOs steered away from polit..."
231,1801,How to Shape Your Culture,,When we compared employees’ views on their org...


In [4]:
text = ''
for i, j in data.iterrows():
    text += j[3] + ' '


Tokenize and process

In [5]:
text_tokens = word_tokenize(text.lower())

In [7]:
tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

In [8]:
unique_words = set(tokens_without_sw)

In [9]:
unique_words = set(tokens_without_sw)
counts = []
for words in unique_words :
    counts.append(tokens_without_sw.count(words))

search = []
text = text.replace("“", '"')
text = text.replace("”", '"')
text = text.replace("’", "'")
text = text.replace("—", "-")

text = text.replace("\'", "'")

for i in unique_words:
    r = ''
    for sentence in text.split('.'):
        if i in sentence:
            r+=sentence + '.\n'
    
    search.append(r)

word_counts = pd.DataFrame(unique_words, columns = ['word'])
word_counts['size'] = counts
word_counts['text'] = search

Remove certain stopwords from the dataframe

In [11]:
stopwords = ['would', 'could', 'well', 'need', 'less', 'said', 'see', 'us', 'might', 'much', 'two', 'get',
            'us', 'many', 'even','’', '“', '”', 'new', 'like', 'good', 'may', 'help' 'three',
            'use', 'others', 'found', 'another',
             'however','long', 'call', 'large', 'used', 'four', 'although', 'makes'] 
for i, j in word_counts.iterrows():
    if j[0] in string.punctuation:
        word_counts = word_counts.drop(i)
    if j[0] in stopwords:
        word_counts = word_counts.drop(i)


In [12]:
r = word_counts.sort_values(by=['size'], ascending = False).head(200)
s = {}
for i,j in r.iterrows():
    s[j[0]] = j[1]

In [13]:
r

,word,size,text
10400,people,2508,Some people blame unconscious bias: High-rank...
12828,company,1975,"Currently the chief talent officer, she leads..."
18388,companies,1527,"(Though once again, there's a discouraging de..."
6029,work,1486,One oft-cited reason why more female executi...
6669,time,1426,") Of course, no CEO-even the executives atop o..."
...,...,...,...
16534,five,229,We often ask senior leaders: Can employees th...
16390,10,228,"Wondering who came in first? Read ""The CEO 10..."
932,opportunities,228,"Meanwhile, glamour work that leads to network..."
20214,done,228,What research have you done on how your femal...


Dump data to a csv file for accessing through R.

In [14]:
r.to_csv("F:/HBR data.csv")